In [1]:
import numpy as np
from collections import defaultdict

from similarity import calculate_similarity_in_batches, vectorizer, tfidf_matrix, docs
from processquery import process


def read_qrels(qrels_file_path):
    qrels = defaultdict(dict)
    query_counts = defaultdict(int)

    with open(qrels_file_path, 'r') as qrels_file:
        for line in qrels_file:
            line_parts = line.strip().split(' ')
            if len(line_parts) != 4:
                print("Invalid line format:", line)
                continue

            query_id, _, doc_id, relevance = line_parts
            query_id = int(query_id)
            doc_idd = doc_id.split('_')[0]

            qrels[query_id][doc_idd] = int(relevance)
            query_counts[query_id] += 1

    return qrels, query_counts


def read_queries(queries_file_path):
    queries = {}
    with open(queries_file_path, 'r') as queries_file:
        for line in queries_file:
            line_parts = line.strip().split(None, 1)
            if len(line_parts) < 2:
                continue
            query_id = int(line_parts[0])
            query = line_parts[1]
            query = process(query)
            queries[query_id] = query

    return queries


In [2]:
def retrieve_similar_documents(query_id, query, qrels, similarity_scores):
    sorted_indices = np.argsort(similarity_scores)[::-1]  # Sort in descending order
    threshold = 0.00000001
    num_relevant = 0
    num_returned = 0
    relevant_documents = []
    precision_list = []
    Ap = []
    reciprocal_rank = 0.0

    for idx in sorted_indices:
        similarity_score = similarity_scores[idx]
        doc_id = list(docs.keys())[idx]
        doc_idd = doc_id.split('_')[0]

        if similarity_score > 0.1:
            if similarity_score >= threshold and doc_idd in qrels.get(query_id, {}):
                num_relevant += 1
                relevant_documents.append(f"Similarity Score: {similarity_score}, Document ID: {doc_id}, Document: {docs[doc_id]}")
                AP = num_relevant / num_returned if num_returned > 0 else 0.0
                Ap.append(AP)

                if num_relevant == 1:  # First relevant document
                    reciprocal_rank = 1 / (num_returned + 1)

            num_returned += 1
            if num_returned >= 10:
                break
            precision = num_relevant / num_returned if num_returned > 0 else 0.0
            precision_list.append(precision)

    Total = len(sorted_indices)

    return num_relevant, num_returned, precision_list, Total, relevant_documents, Ap, reciprocal_rank


def AP(Ap):
    return sum(Ap) / len(Ap) if len(Ap) > 0 else 0.0


def calculate_average_precision(precision_list):
    return sum(precision_list) / len(precision_list) if len(precision_list) > 0 else 0.0


def calculate_recall(num_relevant, query_count):
    return num_relevant / query_count


In [3]:
def process_queries(qrels_file_path, queries_file_path):
    qrels, query_counts = read_qrels(qrels_file_path)
    queries = read_queries(queries_file_path)
    average_precisions = []
    MAP = []
    reciprocal_ranks = []

    for query_id, query in queries.items():
        print("Query ID:", query_id)
        print("Query:", query)

        if query_id not in qrels:
            print("No relevance judgments found for this query.")
            continue

        query_vector = vectorizer.transform([query])
        similarity_scores = calculate_similarity_in_batches(query_vector, tfidf_matrix)

        num_relevant, num_returned, precision_list, Total, relevant_documents, Ap, reciprocal_rank = retrieve_similar_documents(query_id, query, qrels, similarity_scores)

        average_precision = calculate_average_precision(precision_list)
        recall = calculate_recall(num_relevant, query_counts[query_id])

        average_precisions.append(average_precision)
        reciprocal_ranks.append(reciprocal_rank)

        print("Average Precision:", average_precision)
        print("Recall:", recall)
        print("Reciprocal Rank:", reciprocal_rank)

        ap = AP(Ap)  # Calculate AP for the current query
        MAP.append(ap)

        print("AP:", ap)
        print("--------------------------------------")

    mean_ap = np.mean(MAP)
    mean_rr = np.mean(reciprocal_ranks)

    return mean_ap, mean_rr


In [4]:
# Set file paths
qrels_file_path = r"C:\Users\user\Documents\IRSystem1\qrel1.txt"
queries_file_path = r"C:\Users\user\Documents\IRSystem1\queries1.txt"

# Process queries
mean_ap, mean_rr = process_queries(qrels_file_path, queries_file_path)

print("Mean AP:", mean_ap)
print("Mean Reciprocal Rank (MRR):", mean_rr)


Query ID: 3990512
Query: get concentr someth
Average Precision: 0.6652116402116401
Recall: 0.08333333333333333
Reciprocal Rank: 1.0
AP: 1.1666666666666667
--------------------------------------
Query ID: 714612
Query: doesnt water fall earth round
Average Precision: 0.3466931216931217
Recall: 0.13333333333333333
Reciprocal Rank: 0.5
AP: 0.6361111111111111
--------------------------------------
Query ID: 2528767
Query: determin charg iron ion feel3
Average Precision: 0.2058201058201058
Recall: 0.13043478260869565
Reciprocal Rank: 0.25
AP: 0.40277777777777773
--------------------------------------
Query ID: 821387
Query: micehow get rid human
Average Precision: 0.0
Recall: 0.0
Reciprocal Rank: 0.0
AP: 0.0
--------------------------------------
Query ID: 1880028
Query: see leaflet mean pt pregnanc test
Average Precision: 0.5298941798941799
Recall: 0.12903225806451613
Reciprocal Rank: 1.0
AP: 0.7048611111111112
--------------------------------------
Query ID: 4448097
Query: innat immun
Ave